In [1]:
import sqlite3
import pandas as pd

In [105]:
db_bok = "galakse_bok.db"
db_avis = "galakse_avis.db"
db_all = "galakser.db"

In [3]:
def query(db, sql, param=()):
    """ Query a sqlitedatabase with sql and param"""
    with sqlite3.connect(db) as con:
        cur = con.cursor()
        cur.execute(sql, param)
    return cur.fetchall()

# read

In [156]:
bøker = pd.read_csv("../../ngram_2021_coordination/coord-bok.csv", header = None, index_col = 0)

In [82]:
aviser = pd.read_csv("../../ngram_2021_coordination/coord-avis.csv", header = None) 

In [49]:
bøker.columns = ['lang', 'first', 'coord', 'third', 'freq']

In [157]:
bøker

,1,2,3,4
0,,,,
nno,!,og,!,4
nno,!,og,/,4
nno,!,og,60,4
nno,!,og,70,4
nno,!,og,De,4
...,...,...,...,...
nob,",",og,de,3539308
nob,",",og,jeg,3582173
nob,",",og,at,3669361


In [84]:
aviser.columns = ['first', 'coord', 'third', 'freq']

In [91]:
bok = bøker.groupby(['first', 'coord', 'third']).sum('freq').reset_index()

In [108]:
bok_avis = pd.concat([aviser, bok]).groupby(['first', 'coord', 'third']).sum('freq').reset_index()

In [150]:
bok_avis.columns = ['first', 'coord', 'second', 'freq']

bok_avis.to_csv('avis_bøker_tre_kolonner.csv')

# write

In [113]:
def create_database_from_df(db, df):
    """
        db is the name of an empty database, df is a dataframe with columns named first, second, freq
        A table named ngram is created with two secondary tables
    """
    print('creating database from dataframe')
    with sqlite3.connect(db) as con:
        bok_avis[['first','second','freq']].to_sql('ngram', con)
    print('indexing main table')
    query(db, "create index '_ft_' on ngram (first, second)")
    query(db, "create index '_tf_' on ngram (second, first)")
    query(db, "create table firstfreq (first varchar, freq int)")
    query(db, "create table secondfreq (second varchar, freq int)")
    query(db, "insert into firstfreq select first, sum(freq) from ngram group by first")
    query(db, "insert into secondfreq select second, sum(freq) from ngram group by second")
    print('indexing secondary tables')
    query(db, "create index _ff_ on firstfreq (first, freq)")
    query(db, "create index _ftf_ on secondfreq (second, freq)")

In [113]:
def make_galaxy(db):
    print('create galaxies')
    query(db, "create table galaxy (first varchar, second varchar, freq int, pmi float)")
    query(db, "insert into galaxy select  a.first, a.second, sum(a.freq) as f, pow(a.freq,2)*1.0/ (f.freq * t.freq) as pmi from trigrambook as a, firstfreq as f, thirdfreq as t where a.first = f.first and a.second = t.second group by a.first, a.second")
    print('indexin galaxies')
    query(db, "create index _gftp_ on galaxy (first, pmi)")
    query(db, "create index _gtfp_ on galaxy (second, pmi)")

In [143]:
query(db_all, "create table galaxy (first varchar, third varchar, freq int, pmi float)")

[]

In [148]:
query(db_all, "create index _gftpmi_ on galaxy (first, pmi)")
query(db_all, "create index _gtfpmi_ on galaxy (third, pmi)")

[]

In [144]:
query(db_all, "insert into galaxy select  a.first, a.third, sum(a.freq) as f, pow(a.freq,2)*1.0/ (f.freq * t.freq) as pmi from trigrambook as a, firstfreq as f, thirdfreq as t where a.first = f.first and a.third = t.third group by a.first, a.third")

[]

In [114]:
index_db(db_all)

indexing main table
indexing secondary tables


In [122]:
check = lambda x:query(db_all, "select  a.first, a.third, sum(a.freq) as f, pow(a.freq,2)*1.0/ (f.freq * t.freq) as pmi from trigrambook as a, firstfreq as f, thirdfreq as t where a.first = f.first and a.third = t.third and a.first = ? group by a.first, a.third order by pmi desc limit 50",(x,))

In [125]:
check2 = lambda x:query(db_all, "select  a.first, a.third, sum(a.freq), pow(a.freq,2)*1.0/ (f.freq * t.freq) as pmi from trigrambook as a, firstfreq as f, thirdfreq as t where a.first = f.first and a.third = t.third and a.third = ? group by a.first, a.third order by pmi desc limit 50",(x,))

In [117]:
query(db, "select * from galakse where first = 'is' and third = 'snø'")

[(4729431, 'nno', 'is', 'snø', 1435), (4749368, 'nob', 'is', 'snø', 8593)]

In [141]:
check('helse')

[('helse', 'omsorg', 85623, 0.018069636724731684),
 ('helse', 'velferd', 28810, 0.011432093425866532),
 ('helse', 'sosial', 53598, 0.010416164728292683),
 ('helse', 'velvære', 20357, 0.00503860941027041),
 ('helse', 'trivsel', 25729, 0.004762271043125511),
 ('helse', 'livsstil', 11363, 0.0037807265410391023),
 ('helse', 'livskvalitet', 10327, 0.0034290689333111705),
 ('helse', 'førlighet', 2758, 0.0027973745797822223),
 ('helse', 'sunnhet', 4582, 0.0022112458516385055),
 ('helse', 'rus', 6755, 0.0021672395029954435),
 ('helse', 'miljø', 22150, 0.001548633716773125),
 ('helse', 'sykdom', 10077, 0.0014970919809393992),
 ('helse', 'sosialfag', 8879, 0.0011454073958854702),
 ('helse', 'eldreomsorg', 4560, 0.0008093670047256793),
 ('helse', 'sikkerhet', 12205, 0.0006939411295568541),
 ('helse', 'vandel', 3956, 0.0006388881173109643),
 ('helse', 'sosialsektoren', 7871, 0.0006001388750599464),
 ('helse', 'utdanning', 9657, 0.000582694593505117),
 ('helse', 'hygiene', 3205, 0.00051875177053659

In [45]:
query(db, "select  a.first, a.third, a.freq,  f.freq, t.freq from galakse as a, firstfreq as f, thirdfreq  t where a.first = f.first and a.third = t.third and a.first = 'spiser' order by a.freq desc limit 50 ")

[('spiser', 'drikker', 9166, 17552, 49439),
 ('spiser', 'sover', 1664, 17552, 30979),
 ('spiser', 'spiser', 673, 17552, 47566),
 ('spiser', 'hvordan', 360, 17552, 701901),
 ('spiser', 'hvor', 312, 17552, 827034),
 ('spiser', 'lever', 289, 17552, 45866),
 ('spiser', 'snakker', 248, 17552, 73789),
 ('spiser', 'koser', 221, 17552, 4015),
 ('spiser', 'blir', 213, 17552, 522969),
 ('spiser', 'hva', 180, 17552, 727677),
 ('spiser', 'ikke', 178, 17552, 2699912),
 ('spiser', 'drikker', 175, 17552, 49439),
 ('spiser', 'prater', 175, 17552, 18255),
 ('spiser', 'vokser', 159, 17552, 18538),
 ('spiser', 'ser', 157, 17552, 624375),
 ('spiser', 'når', 149, 17552, 825050),
 ('spiser', 'går', 145, 17552, 427766),
 ('spiser', 'er', 119, 17552, 1976015),
 ('spiser', 'har', 108, 17552, 1724528),
 ('spiser', 'det', 101, 17552, 12381634),
 ('spiser', 'hviler', 95, 17552, 10433),
 ('spiser', 'tar', 87, 17552, 295266),
 ('spiser', 'gjør', 80, 17552, 283196),
 ('spiser', 'legger', 80, 17552, 184058),
 ('spise